## Data Analysis and the Perception of Time

The purpose of this analysis is to determine if there is a relationship between the change in the S&P 500 value and the day of the week.

$ \large\frac{\text{Closing price on a given day}}{\text{Closing price on exactly 1 weekday before}} \small- 1 $ 

Data are excluded if the previous trading day is not exactly 1 weekday before e.g. Monday -> Thursday, Friday -> Wednesday. 

In [5]:
import numpy as np
import pandas as pd
import datetime

# input daily historical data (10 years) on the SPY etf from Yahoo! Finance. 
df = pd.read_csv("SPY.csv")
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-10-31,141.850006,142.029999,140.679993,141.350006,117.104858,103438500
1,2012-11-01,141.649994,143.009995,141.520004,142.830002,118.330986,100995600
2,2012-11-02,143.679993,143.720001,141.410004,141.559998,117.278770,137702200
3,2012-11-05,141.350006,142.169998,140.929993,141.850006,117.519066,98378500
4,2012-11-06,142.279999,143.520004,142.130005,142.960007,118.438698,107068100
...,...,...,...,...,...,...,...
2511,2022-10-21,365.119995,374.799988,363.540009,374.290009,374.290009,130897700
2512,2022-10-24,375.890015,380.059998,373.109985,378.869995,378.869995,85436900
2513,2022-10-25,378.790009,385.250000,378.670013,384.920013,384.920013,78846300
2514,2022-10-26,381.619995,387.579987,381.350006,382.019989,382.019989,104087300


___
"Adj Close" is the closing price that has been adjusted for splits and dividends.

In [6]:
# retrieve the relevant columns 
df = df.loc[:, ["Date", "Adj Close"]]

# convert the "Adj Close" column to an array for faster operations 
cp = df["Adj Close"].to_numpy()

# percentage change in closing price
pct = cp[1:] / cp[:-1]
pct -= 1

# insert a NaN value into the first row as it is not calculated
pct = np.insert(pct, 0, np.nan)

# convert returns array to Series for concatenation
pct = pd.Series(pct, name="pct_change")

# concat the Series back to the dataframe 
df = pd.concat([df, pct], axis=1)

___
Add columns for the day names and their previous weekdays to filter the data later. 

In [7]:
# change dtype of "Date" column to datetime 
df["Date"] = pd.to_datetime(df["Date"])

# add column of day names 
df = df.assign(day=df["Date"].dt.strftime("%a"))

# add column of previous weekday names 
df = df.assign(prev_wkday=df["day"].shift(1))

df

,Date,Adj Close,pct_change,day,prev_wkday
0,2012-10-31,117.104858,NaN,Wed,None
1,2012-11-01,118.330986,0.010470,Thu,Wed
2,2012-11-02,117.278770,-0.008892,Fri,Thu
3,2012-11-05,117.519066,0.002049,Mon,Fri
4,2012-11-06,118.438698,0.007825,Tue,Mon
...,...,...,...,...,...
2511,2022-10-21,374.290009,0.024301,Fri,Thu
2512,2022-10-24,378.869995,0.012236,Mon,Fri
2513,2022-10-25,384.920013,0.015969,Tue,Mon
2514,2022-10-26,382.019989,-0.007534,Wed,Tue


___
Finally, filter and aggregate the data.

In [8]:
days = ["Mon", "Tue", "Wed", "Thu", "Fri"]

agg_df = pd.DataFrame([])

for i, day in enumerate(days):
    # filter the data
    fil_df = df[(df["day"] == day) & (df["prev_wkday"] == days[i - 1])]

    # aggregate the data
    fil_df = fil_df.agg({"pct_change": ["mean", "std", "max", "min"]})

    fil_df = fil_df.rename(columns={"pct_change": day})

    # combine data for output
    agg_df = pd.concat([agg_df, fil_df], axis=1)

agg_df = agg_df * 100

agg_df = agg_df.round(2).astype(str) + "%"

agg_df

,Mon,Tue,Wed,Thu,Fri
mean,0.02%,0.1%,0.07%,0.02%,0.04%
std,1.2%,1.05%,1.01%,1.08%,1.08%
max,6.72%,9.06%,4.2%,5.84%,8.55%
min,-10.94%,-4.35%,-5.06%,-9.57%,-4.31%
